In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, SimpleRNN, LSTM, Dropout, Bidirectional
import keras
from keras.preprocessing import sequence

import copy
import random


from py4j.java_gateway import JavaGateway
from collections import deque


In [ ]:
trace1 = ['A', 'B', 'D', 'E', 'I']
trace2 = ['A', 'C', 'D', 'G', 'H', 'F', 'I']
trace3 = ['A', 'C', 'G', 'D', 'H', 'F', 'I']
trace4 = ['A', 'C', 'H', 'D', 'F', 'I']
trace5 = ['A', 'C', 'D', 'H', 'F', 'I']


traces = [trace1, trace2, trace3, trace4, trace5]
frequencies = [1207, 145, 56, 23, 28]

GT_log = []
for f in range(0, len(frequencies)):
    print(traces[f], frequencies[f])
    for i in range(0, frequencies[f]):
        GT_log.append(traces[f])
        
random.shuffle(GT_log)

In [ ]:
def check(trace, log):
    if trace in log:
        return True
    else:
        return False


In [ ]:
def PARTget_voc(log):
    outputset1 = set([])
    #outputset2 = set([]) #for voc with only log2 for creating antilog
    for trace in log:
        for act in trace:
            outputset1.add(act)
    voc = list(outputset1)
    return voc
    

In [ ]:
def PARTadd_noise_name(logdummy, voc, percentage):
    changelog = []
    amount_of_noisy_traces = int(percentage * len(logdummy))
    noisy_examples = random.sample(range(0, len(logdummy)), amount_of_noisy_traces)
    for i in range(0, len(logdummy)):
        trace = copy.deepcopy(logdummy[i])
        if i in noisy_examples:
            random_act = random.randint(0, len(logdummy[i]) - 1)
            new = random.choice(voc)
            #print(new)
            trace[random_act] = new
            done = check(trace, logdummy)
            while done == True:
                #print("double trouble")
                random_act = random.randint(0, len(logdummy[i]) - 1)
                new = random.choice(voc)
                #print(new)
                trace[random_act] = new
                done = check(trace, logdummy)
            changelog.append(trace)
        else:
            changelog.append(trace)
    return changelog

def PARTadd_noise_order(logdummy, percentage):
    changelog = []
    amount_of_noisy_traces = int(percentage * len(logdummy))
    noisy_examples = random.sample(range(0, len(logdummy)), amount_of_noisy_traces)
    for i in range(0, len(logdummy)):
        trace = copy.deepcopy(logdummy[i])
        if i in noisy_examples:
            random_act = random.randint(0, len(logdummy[i]) - 1)
            random_act2 = random.randint(0, len(logdummy[i]) - 1)
            first = copy.copy(logdummy[i][random_act])
            second = copy.copy(logdummy[i][random_act2])
            #print(first, second)
            trace[random_act] = second
            trace[random_act2] = first
            done = check(trace, logdummy)
            while done == True:
                random_act = random.randint(0, len(logdummy[i]) - 1)
                random_act2 = random.randint(0, len(logdummy[i]) - 1)
                first = copy.copy(logdummy[i][random_act])
                second = copy.copy(logdummy[i][random_act2])
                #print(first, second)
                trace[random_act] = second
                trace[random_act2] = first
                done = check(trace, logdummy)
            changelog.append(trace)
        else:
            changelog.append(trace)
    return changelog



def PARTadd_noise_delete(logdummy, percentage):
    changelog = []
    amount_of_noisy_traces = int(percentage * len(logdummy))
    noisy_examples = random.sample(range(0, len(logdummy)), amount_of_noisy_traces)
    for i in range(0, len(logdummy)):
        trace = copy.deepcopy(logdummy[i])
        if i in noisy_examples:
            random_act = random.randint(0, len(logdummy[i]) - 1)
            del trace[random_act]
            changelog.append(trace)
        else:
            changelog.append(trace)
    return changelog

In [ ]:
def PARTadd_noise(logdummy, percentage):
    l = copy.deepcopy(logdummy)
    voc = PARTget_voc(logdummy)
    l1 = PARTadd_noise_name(l, voc, percentage)
    #l2 = PARTadd_noise_order(l, percentage)
    #l3 = PARTadd_noise_delete(l2, percentage)
    return(l1)

In [ ]:
class OutputGetter(deque):
    def appendleft(self, line):
        print(line)
        super().appendleft(line)
        
def remove_stuff(log): #removes short traces
    #new_log = [i for i in log if len(i) > 5]
    new_log = [i for i in log if len(i) > 1]
    return new_log
    
        
def remove_plus(dummylog):
    newlog = copy.deepcopy(dummylog)
    newlog = [[item.replace('+', '') for item in lst] for lst in newlog]
    return newlog
       
def fix_duplicates(dummylog):
    newlog = []
    for trace in dummylog:
        newtrace = trace[::2]
        newlog.append(newtrace)
    return newlog

def check_duplicates(dummylog): #not a good test but just to be sure
    if len(dummylog) > 1 and dummylog[0][0] == dummylog[0][1] and dummylog[1][0] == dummylog[1][1]:
        return True
    elif len(dummylog) == 1 and dummylog[0][0] == dummylog[0][1]:
        return True
    else:
        return False

In [ ]:
def get_model_log(modelfilename, size):
    gateway = JavaGateway.launch_gateway(classpath="./jars/*", 
                                         redirect_stdout=OutputGetter())
    petri_and_marking = gateway.jvm.org.processmining.plugins.kutoolbox.utils.ImportUtils.openPetrinet(
        gateway.jvm.java.io.File(modelfilename)
    )
    if not petri_and_marking[1].size():
        print("Creating initial marking")
        petri_and_marking[1] = gateway.jvm.org.processmining.plugins.kutoolbox.utils.PetrinetUtils. \
            getInitialMarking(petri_and_marking[0])

    petri_and_marking[1].size()
    settings = gateway.jvm.org.processmining.plugins.loggenerator.utils.GeneratorSettings()
    simulator = gateway.jvm.org.processmining.plugins.loggenerator.utils.GeneratorSettings.SimulationAlgorithm

    # See https://github.com/Macuyiko/processmining-prom/blob/master/loggenerator/
    #             org/processmining/plugins/loggenerator/utils/GeneratorSettings.java

    for t in petri_and_marking[0].getTransitions().iterator():
        isInvisible = t.getLabel() == "" or t.isInvisible()
        label = t.getLabel()
        isTextInvisible = label.startswith("inv_") or "$" in label
        mapped = label.replace("+complete", "").replace("\\ncomplete", "").replace("\\n", "") \
            if not isInvisible and not isTextInvisible else ""
        arr = gateway.new_array(gateway.jvm.int, 4)
        arr[0], arr[1], arr[2], arr[3] = 60, 0, 60, 0
        settings.getTransitionNames().put(t, mapped)
        settings.getTransitionWeights().put(t, 10)
        settings.getTransitionTimings().put(t, arr)
        #print(t.getId(), ':', label, '--->', mapped)

        settings.setNrTraces(size)
        settings.setRandomMinInGroup(1)
        settings.setRandomMaxInGroup(1)
        settings.setMustReachEnd(True)
        settings.setMustConsumeAll(False)
        settings.setMaxTimesMarkingSeen(7)

        settings.setSimulationMethod(simulator.Random) # Random, Complete, or Distinct

        # settings.setRestartAfter(1000)
        # settings.setSkipChance(0.85)
        # settings.setAlsoConsiderPartial(False)
    xlog = gateway.jvm.org.processmining.plugins.loggenerator.PetriNetLogGenerator.generate(
        petri_and_marking[0], 
        petri_and_marking[1], 
        settings, 
        #None # or for output: gateway.jvm.org.processmining.plugins.kutoolbox.eventlisteners.PluginEventListenerCLI()
        gateway.jvm.org.processmining.plugins.kutoolbox.eventlisteners.PluginEventListenerCLI()
    )

    temp_log = []
    for trace in xlog:
        #print([gateway.jvm.org.deckfour.xes.extension.std.XConceptExtension.instance().extractName(e) for e in trace])
        temp_log.append([gateway.jvm.org.deckfour.xes.extension.std.XConceptExtension.instance().extractName(e) for e in trace])

    model_log = remove_stuff(temp_log)


    if check_duplicates(model_log) == True:
        print('Fixing duplicates')
        model_log = fix_duplicates(model_log)
    return(model_log)

In [ ]:
#model_log = get_model_log("Model1.pnml", 1500)

In [ ]:
def get_voc(log1, log2):
    log = log1 + log2
    outputset1 = set([])
    #outputset2 = set([]) #for voc with only log2 for creating antilog
    for trace in log:
        for act in trace:
            outputset1.add(act)
    voc = list(outputset1)
    return voc
    


def add_noise_name(logdummy, voc):
    changelog = []
    for i in range(0, len(logdummy)):
        trace = copy.deepcopy(logdummy[i])
        random_act = random.randint(0, len(logdummy[i]) - 1)
        new = random.choice(voc)
        #print(new)
        trace[random_act] = new
        changelog.append(trace)
    return changelog

def add_noise_order(logdummy):
    changelog = []
    for i in range(0, len(logdummy)):
        trace = copy.deepcopy(logdummy[i])
        random_act = random.randint(0, len(logdummy[i]) - 1)
        random_act2 = random.randint(0, len(logdummy[i]) - 1)
        first = copy.copy(logdummy[i][random_act])
        second = copy.copy(logdummy[i][random_act2])
        #print(first, second)
        trace[random_act] = second
        trace[random_act2] = first
        changelog.append(trace)
    return changelog
    

#SHOULD CHECK IF WE WANT TO DO THIS!!!!!!
def delete_not_new(log, antilog):
    unique = []
    for trace in log:
        if trace not in unique:
            unique.append(trace)
        else:
            continue
    #print(unique)
    new_antilog = []
    for trace in antilog:
        if trace not in unique:
            new_antilog.append(trace)
        else:
            continue
    return(new_antilog)


def get_antilog_noise(logdummy, voc):
    l = copy.deepcopy(logdummy)
    l1 = add_noise_name(l, voc)
    l2 = add_noise_order(l1)
    antilog = delete_not_new(logdummy, l2)
    return(antilog)
    
def get_antilog_random(log, voc):
    min_length = len(min(log, key=len))
    max_length = len(max(log, key=len)) #now we just take random between min and max, should probably be changed to normal distribution
    length_antilog = len(log) #for size antilog = size log, but can be altered
    antilog = []
    for i in range(0, length_antilog):
        size = random.randint(min_length,max_length)
        antitrace = []
        for j in range(0, size):
            index = random.randint(0,len(voc)-1)
            antitrace.append(voc[index])
        antilog.append(antitrace)
    return(antilog)

def padding(dumlog, max_length, pad):
    log = copy.deepcopy(dumlog)
    newlog =  []
    for trace in log:
        if len(trace)>max_length:
            while len(trace)>max_length:
                trace.pop()
        elif len(trace)<max_length:   
            while len(trace)<max_length:
                trace.append(pad)
        newlog.append(trace)
    return newlog

def get_batches(log): #get same sized batches in list of numpy multidim arrays
    #log is log WITH label!
    min_length = len(min([item[0] for item in log], key=len))
    max_length = len(max([item[0] for item in log], key=len))
    print(min_length, max_length)
    Xbatches = []
    Ybatches = []
    for size in range(min_length, max_length+1):
        print(size)
        listX = []
        listY = []
        for i in range(0, len(log)):
            if len(log[i][0]) == size:
                listX.append(log[i][0])
                listY.append(log[i][1])
        if len(listY) == 0:
            print(size, "No traces")
            continue
        arrayX = np.array(listX)
        #arrayX = np.expand_dims(arrayX, axis=2)
        #print(arrayX)
        arrayY = np.array(listY)
        Xbatches.append(arrayX)
        Ybatches.append(arrayY)   
    return(Xbatches, Ybatches)
        
    
    



In [ ]:
def get_model(dumlog2, dumantilog, voc):

    maximumlength = len(max(dumlog2, key=len))

    #dumlog2 = padding(dumlog2, maximumlength, "Z")
    #dumantilog = padding(dumantilog, maximumlength, "Z")

    log2 = copy.deepcopy(dumlog2)
    antilog = copy.deepcopy(dumantilog)
    #voc.append("Z")
    label_encoder = LabelEncoder() #label encoder 
    label_encoder.fit(voc)
    
    #list of arrays (traces) with encoded activities as numpy array
    trainlog = []
    for i in range(len(log2)):
        dummytrace = label_encoder.transform(log2[i])
        trace = dummytrace.tolist()
        trainlog.append([trace,1]) #1 = label true trace
    for i in range(len(antilog)):
        dummytrace = label_encoder.transform(antilog[i])
        trace = dummytrace.tolist()
        trainlog.append([trace,0]) #0 = label antitrace
    
    X = []
    Y = []
    
    for trace in trainlog:
        X.append(trace[0])
        Y.append(trace[1])
    X = sequence.pad_sequences(X, maxlen=maximumlength)
    X = np.array(X)
    Y = np.array(Y)
    
    print("Fixed preprocessing")
    
    model = Sequential()
    model.add(Embedding(input_dim=len(voc),output_dim=4, input_length=maximumlength))
    #model.add(LSTM(units=32))
    model.add(Bidirectional(SimpleRNN(units=8)))
    #model.add(Bidirectional(LSTM(units=8)))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['acc'])
    hist = model.fit(X, Y, batch_size=64, epochs=40, verbose = 0, validation_split = 0.2)
    print("training done")
    return(model, maximumlength, label_encoder)


In [ ]:
def get_fit(model, dumlog1, maximumlength, label_encoder):
    #dumlog1 = padding(dumlog1, maximumlength, "Z")
    log1 = copy.deepcopy(dumlog1)
    testlog = []
    for trace in log1:
        dummytrace = label_encoder.transform(trace)
        testlog.append(dummytrace)
    testlog = sequence.pad_sequences(testlog, maxlen=maximumlength)
    testlog = np.array(testlog)
    squeezed = model.predict(testlog)
    fit1 = np.average(squeezed)
    count_0 = 0
    count_1 = 0
    for pred in squeezed:
        if pred < 0.5:
            count_0 = count_0 + 1
        if pred >= 0.5:
            count_1 = count_1 + 1
    #print("Amount predicted true: ", count_0, ". Amount predicted antitrace: ", count_1)
    fitness = count_1/(count_1 + count_0)
    return(fit1, fitness)

In this test we are training the discriminator on the model log ONCE.
We then add different levels of noise to the GT log and put that trough the discriminator.

In [ ]:
def test_fitness(filename, modelsize, GT_log, noises): 
    model_log = get_model_log(filename, modelsize)
    voc = get_voc(GT_log, model_log)
    antilog = get_antilog_random(model_log, voc)
    
    model, maxlength, enc = get_model(model_log, antilog, voc)
    fit1list = []
    fit2list = []
    for n in noises:
        noisy_log = PARTadd_noise(GT_log, n)
        print("NOISE", n)
        fit1, fit2 = get_fit(model, noisy_log, maxlength, enc)
        print("PROB: ", fit1, " COUNT: ", fit2)
        fit1list.append(fit1)
        fit2list.append(fit2)
    return(fit1list, fit2list)

In this test we are training the discriminator on the GT log ONCE.
We then add different levels of noise to the model log and put that trough the discriminator.

In [ ]:
def test_precision(filename, modelsize, GT_log, noises): 
    model_log = get_model_log(filename, modelsize)
    voc = get_voc(model_log, GT_log)
    antilog = get_antilog_random(GT_log, voc)
    
    model, maxlength, enc = get_model(GT_log, antilog, voc)
    fit1list = []
    fit2list = []
    for n in noises:
        noisy_log = PARTadd_noise(model_log, n)
        print("NOISE", n)
        fit1, fit2 = get_fit(model, noisy_log, maxlength, enc)
        print("PROB: ", fit1, " COUNT: ", fit2)
        fit1list.append(fit1)
        fit2list.append(fit2)
    return(fit1list, fit2list)

In [ ]:
noises = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [ ]:
fitlist1, fitlist2 = test_fitness("Model1.pnml", 1500, GT_log, noises)

In [ ]:
preclist1, preclist2 = test_precision("Model1.pnml", 1500, GT_log, noises)

In [ ]:
print(GT_log)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
'''
plt.plot(noises, fitlist1, label="Recall prob.")
plt.plot(noises, fitlist2, label="Recall count")
plt.plot(noises, preclist1, label="Precision prob.")
plt.plot(noises, preclist2, label="Precision count")
plt.xlabel('Fraction of instances with noise')
plt.ylabel('Recall/Precision')
plt.title('Recall/Precision doing random swaps')
plt.legend()
plt.savefig("swap.pdf")
plt.show()
'''

In [ ]:

plt.plot(noises, fitlist1, label="Recall prob.")
plt.plot(noises, fitlist2, label="Recall count")
plt.plot(noises, preclist1, label="Precision prob.")
plt.plot(noises, preclist2, label="Precision count")
plt.xlabel('Fraction of instances with noise')
plt.ylabel('Recall/Precision')
plt.title('Recall/Precision doing random replacements')
plt.legend()
plt.savefig("replacement.pdf")
plt.show()


In [ ]:
print(GT_log)

In [ ]:
#model_log = get_model_log("Model1.pnml", 1500)
voc = get_voc(GT_log, GT_log)
antilog = get_antilog_random(GT_log, voc)
#antilog = get_antilog_noise(GT_log, voc)
model, maxlength, enc = get_model(GT_log, antilog, voc)

In [ ]:
fitting = [['A', 'B', 'D', 'E', 'I'], ['A', 'B', 'D', 'E', 'I'], ['A', 'B', 'D', 'E', 'I']]

fitting2 = [['A', 'C', 'H', 'D', 'F', 'I'], ['A', 'C', 'H', 'D', 'F', 'I'], ['A', 'C', 'H', 'D', 'F', 'I']]

nonfitting = [['A', 'B', 'E', 'D', 'I'], ['A', 'B', 'E', 'D', 'I'], ['A', 'B', 'E', 'D', 'I']]

alsononfitting =  [['A', 'D', 'B', 'E', 'I'], ['A', 'D', 'B', 'E', 'I'], ['A', 'D', 'B', 'E', 'I']]

nonfitting3 =  [['A', 'B', 'E', 'I'], ['A', 'B', 'E', 'I'], ['A', 'B', 'E', 'I']]

In [ ]:
fit1, bla1 = get_fit(model, fitting, maxlength, enc)

fit2, bla2 = get_fit(model,fitting2,maxlength,enc)

nonfit1, nonfit2 = get_fit(model, nonfitting, maxlength, enc)

alsononfit1, alsononfit2 = get_fit(model, alsononfitting, maxlength, enc)

nonfit1three, nonfit2three = get_fit(model, nonfitting3, maxlength, enc)

In [ ]:
print(fit1, bla1)
print(fit2, bla2)
print(nonfit1, nonfit2)
print(alsononfit1, alsononfit2)
print(nonfit1three, nonfit2three)

In [ ]:
trace1 = ['A', 'B', 'D', 'E', 'I']
trace2 = ['A', 'C', 'D', 'G', 'H', 'F', 'I']
trace3 = ['A', 'C', 'G', 'D', 'H', 'F', 'I']
trace4 = ['A', 'C', 'H', 'D', 'F', 'I']
trace5 = ['A', 'C', 'D', 'H', 'F', 'I']


In [ ]:
def get_fit_distribution(model, dumlog1, maximumlength, label_encoder):
    #dumlog1 = padding(dumlog1, maximumlength, "Z")
    log1 = copy.deepcopy(dumlog1)
    testlog = []
    for trace in log1:
        dummytrace = label_encoder.transform(trace)
        testlog.append(dummytrace)
    testlog = sequence.pad_sequences(testlog, maxlen=maximumlength)
    testlog = np.array(testlog)
    squeezed = model.predict(testlog)
    fit1 = np.average(squeezed)
    count_0 = 0
    count_1 = 0
    for pred in squeezed:
        if pred < 0.5:
            count_0 = count_0 + 1
        if pred >= 0.5:
            count_1 = count_1 + 1
    #print("Amount predicted true: ", count_0, ". Amount predicted antitrace: ", count_1)
    fitness = count_1/(count_1 + count_0)
    return(squeezed, fit1, fitness)

In [ ]:
model_log = get_model_log("Model1.pnml", 1500)
voc = get_voc(model_log, GT_log)
#antilog = get_antilog_random(GT_log, voc)
antilog = get_antilog_noise(model_log, voc)
model, maxlength, enc = get_model(model_log, antilog, voc)



In [ ]:

noisy_log = PARTadd_noise(GT_log, 0.75)

In [ ]:
dist, fitprob, fitcount = get_fit_distribution(model, noisy_log, maxlength, enc)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# matplotlib histogram
plt.hist(dist, color = 'blue', edgecolor = 'black',
         bins = 40)

# seaborn histogram
sns.distplot(dist, hist=True, kde=False, 
             bins=40, color = 'blue',
             hist_kws={'edgecolor':'black'})
# Add labels
plt.title('Distribution Recall values of different instances')
plt.xlabel('Probability trace')
plt.ylabel('#instances')
plt.savefig("Distribution.pdf")

In [ ]:
print(fitprob, fitcount)

In [ ]:
model.summary()

In [ ]:
print(dist)